# Notebook 08: Gemini LLM Reasoning Module

Purpose:
- Generate structured, human-readable reasoning using Gemini
- Respect Supervisor decisions and confidence levels
- Convert analytics + behavioral signals into cautious suggestions
- Allow natural, engaging responses using guided emoji autonomy
- Prevent overconfident or unsafe AI behavior

In [1]:
# !pip install google-generativeai python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from google import genai

In [3]:
load_dotenv()

GEMINI_API_KEY = os.getenv("Google_Gemini_Api_Key")

if not GEMINI_API_KEY:
    raise ValueError("Gemini API key not found in .env file")

In [4]:
client = genai.Client(api_key=GEMINI_API_KEY)

In [5]:
supervisor_decision = {
    "status": "ALLOW",
    "confidence": "LOW",
    "warnings": [
        "Productivity instability detected",
        "Conflict: productivity high but user reports fatigue"
    ]
}

In [6]:
analytics_summary = {
    "avg_work_minutes": 421,
    "avg_leisure_minutes": 255,
    "avg_sleep_minutes": 426,
    "avg_exercise_minutes": 64,
    "mean_productivity": 82
}

In [7]:
user_profile = {
    "education": "MCA",
    "goal": "Become an ML Engineer",
    "priority": "Learning + Health"
}

In [8]:
def build_reasoning_prompt(profile, analytics, supervisor):
    warnings_text = (
        "\n".join(f"- {w}" for w in supervisor["warnings"])
        if supervisor["warnings"]
        else "None"
    )

    prompt = f"""
You are NEEL, a cautious and responsible AI assistant.

USER CONTEXT:
- Education: {profile['education']}
- Goal: {profile['goal']}
- Priority: {profile['priority']}

ANALYTICS SUMMARY:
- Average Work Time: {analytics['avg_work_minutes']} minutes/day
- Average Leisure Time: {analytics['avg_leisure_minutes']} minutes/day
- Average Sleep Time: {analytics['avg_sleep_minutes']} minutes/day
- Average Exercise Time: {analytics['avg_exercise_minutes']} minutes/day
- Mean Productivity Score: {analytics['mean_productivity']}

SUPERVISOR STATUS:
- Confidence Level: {supervisor['confidence']}
- Warnings:
{warnings_text}

INSTRUCTIONS:
- Do NOT give medical or absolute advice
- Be cautious if confidence is LOW
- Explicitly acknowledge uncertainty
- Do NOT mention ML models or scores
- Keep the tone professional and calm
- Respond ONLY in the format below

STYLE GUIDELINES:
- You may use at most one relevant emoji to improve warmth or clarity
- Emoji usage should always remain subtle, professional, and context-aware
- When confidence is MEDIUM or HIGH, light emoji use is encouraged if it supports clarity or reassurance
- In serious or reflective situations, emojis should be used sparingly and only if they add calm or balance
- Avoid playful or energetic emojis in sensitive topics (e.g., fatigue, stress, burnout)
- Do not use emojis when confidence is LOW unless they convey calm neutrality
- Preferred emojis include: 🧠 ⚖️ 🌱 ⏳
- Emojis are optional and should never distract from the message

RESPONSE FORMAT:
OBSERVATION:
REASONING:
SUGGESTION:
CONFIDENCE NOTE:
"""
    return prompt

In [9]:
if supervisor_decision["status"] == "BLOCK":
    print("Supervisor blocked reasoning. No LLM response generated.")

In [10]:
if supervisor_decision["status"] == "ALLOW":
    prompt = build_reasoning_prompt(
        user_profile,
        analytics_summary,
        supervisor_decision
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    print(response.text)

OBSERVATION:
Your analytics indicate a substantial dedication to work and learning activities, with an average of 421 minutes per day. Concurrently, there is a reported experience of fatigue, which appears to be in contrast with the observed high output during work periods. Furthermore, there is an indication of potential instability in your productivity patterns.

REASONING:
The reported fatigue, despite significant work engagement, suggests that the current allocation of time, while productive in output, might be exerting a notable impact on your overall well-being. While your goal of becoming an ML Engineer requires dedicated learning, balancing this with your health priorities is essential for sustained progress. My current analytical insights are limited regarding the specific underlying factors contributing to this fatigue and the perceived productivity instability.

SUGGESTION:
It may be beneficial to carefully review the intensity and structure of your daily work and learning s

## LLM Reasoning Summary

- Gemini is invoked only after Supervisor approval
- Responses are structured and explainable
- Emojis are optional and governed by style rules
- Confidence level controls tone and caution
- LLM acts as a reasoning assistant, not a decision-maker